# Logic Gate Neural Network with Genetic Algorithm

In this notebook, we build a simple neural network composed entirely of logic gates (AND, OR, XOR, NOT).  
Each neuron applies a logic operation to its binary inputs, and the network is organized in layers.

Once a target logic gate neural network is created, we use a **Genetic Algorithm (GA)** to evolve a new network that replicates the target's internal structure.  
The goal is for the GA to discover a network with the same configuration of logic gates and connections as the original.

This approach demonstrates how evolutionary techniques can be used to reverse-engineer or approximate logical architectures without access to internal parameters—just using their behavior and structure.


In [ ]:
import numpy as np
import copy

## Logic Gate Functions

In [ ]:
def AND(a, b):
    return a & b

def OR(a, b):
    return a | b

def XOR(a, b):
    return a ^ b

def NOT(a):
    return ~a & 1

In [ ]:
gate_functions = {
    "AND": AND,
    "OR": OR,
    "XOR": XOR,
    "NOT": NOT,
}

## LogicGateNeuron Class

Represents a single neuron that applies a logic gate operator to its inputs.

- Stores the operator type (AND, OR, XOR, NOT).
- Holds indices of inputs it uses.
- Computes output by applying the operator on selected inputs.
- Can mutate its operator and inputs randomly.

In [ ]:
class LogicGateNeuron:
    def __init__(self, operator="AND", inputs=None):
        self.operator = operator
        self.inputs = inputs if inputs is not None else []

    def compute(self, input_values):
        if self.operator == "NOT":
            return NOT(input_values[self.inputs[0]])
        elif len(self.inputs) == 2:
            a, b = input_values[self.inputs[0]], input_values[self.inputs[1]]
            return gate_functions[self.operator](a, b)
        else:
            raise ValueError(f"Invalid number of inputs for operator {self.operator}")

    def mutate(self, num_inputs):
        if num_inputs == 1:
            self.operator = "NOT"
            self.inputs = [0]
        else:
            self.operator = np.random.choice(list(gate_functions.keys()))
            num_selected_inputs = 1 if self.operator == "NOT" else 2
            self.inputs = np.random.choice(range(num_inputs), size=num_selected_inputs, replace=False).tolist()

    def __repr__(self):
        return f"{self.operator}({', '.join(map(str, self.inputs))})"


## LogicGateLayer Class

Represents a layer of logic gate neurons.

- Initialized with a number of neurons and inputs.
- Each neuron randomly selects two inputs initially.
- Computes outputs by applying each neuron's logic gate.
- Can mutate all neurons in the layer.

In [ ]:
class LogicGateLayer:
    def __init__(self, num_neurons, num_inputs):
        self.neurons = [LogicGateNeuron(inputs=np.random.choice(range(num_inputs), size=2, replace=False).tolist())
                        for _ in range(num_neurons)]

    def compute(self, input_values):
        return np.array([neuron.compute(input_values) for neuron in self.neurons])

    def mutate(self, num_inputs):
        for neuron in self.neurons:
            neuron.mutate(num_inputs)

    def __repr__(self):
        return f"Layer: [{', '.join(map(str, self.neurons))}]"

## LogicGateNetwork Class

Represents the full network composed of multiple layers.

- Layers are created sequentially, each taking inputs from the previous layer.
- The first layer takes the network input.
- The network propagates input through all layers to produce output.
- Supports mutation of all layers to randomly change neuron gates and inputs.

In [ ]:
class LogicGateNetwork:
    def __init__(self, layer_sizes, input_size):
        self.layers = []
        current_input_size = input_size
        for size in layer_sizes:
            self.layers.append(LogicGateLayer(size, current_input_size))
            current_input_size = size

    def compute(self, input_values):
        for layer in self.layers:
            input_values = layer.compute(input_values)
        return input_values

    def mutate(self):
        """Mutuje całą sieć."""
        for i, layer in enumerate(self.layers):
            num_inputs = len(self.layers[i - 1].neurons) if i > 0 else len(self.layers[0].neurons[0].inputs)
            layer.mutate(num_inputs)

    def __repr__(self):
        return '\n'.join([f"Layer {i}: {layer}" for i, layer in enumerate(self.layers)])

## Data Generation and Mutation

- Generates random binary inputs and computes corresponding outputs using the network.
- Prints initial network structure and generated data samples.
- Applies mutation to the network (random changes to neurons).
- Prints network structure and outputs after mutation to observe changes.

In [ ]:
def generate_data(network, num_samples, input_size):
    inputs = [np.random.randint(0, 2, size=input_size) for _ in range(num_samples)]
    outputs = [network.compute(inp) for inp in inputs]
    return inputs, outputs

In [ ]:
network = LogicGateNetwork(layer_sizes=[4, 3, 1], input_size=3)

print("Initial network structure:")
print(network)

num_samples = 10
input_size = 3
inputs, outputs = generate_data(network, num_samples, input_size)

print("\nGenerated data:")
for i, (inp, out) in enumerate(zip(inputs, outputs)):
    print(f"Sample {i + 1}: Input: {inp}, Output: {out}")

network.mutate()
print("\nNetwork structure after mutation:")
print(network)

outputs_after_mutation = [network.compute(inp) for inp in inputs]
print("\nOutputs after mutation:")
for i, (inp, out) in enumerate(zip(inputs, outputs_after_mutation)):
    print(f"Sample {i + 1}: Input: {inp}, Output after mutation: {out}")

Initial network structure:
Layer 0: Layer: [AND(0, 1), AND(2, 0), AND(1, 0), AND(1, 0)]
Layer 1: Layer: [AND(1, 0), AND(2, 0), AND(3, 0)]
Layer 2: Layer: [AND(1, 0)]

Generated data:
Sample 1: Input: [0 1 0], Output: [0]
Sample 2: Input: [1 1 1], Output: [1]
Sample 3: Input: [1 1 0], Output: [0]
Sample 4: Input: [0 0 1], Output: [0]
Sample 5: Input: [1 0 1], Output: [0]
Sample 6: Input: [1 0 1], Output: [0]
Sample 7: Input: [1 1 0], Output: [0]
Sample 8: Input: [1 0 0], Output: [0]
Sample 9: Input: [0 1 0], Output: [0]
Sample 10: Input: [1 1 1], Output: [1]

Network structure after mutation:
Layer 0: Layer: [AND(0, 1), NOT(0), OR(1, 0), AND(0, 1)]
Layer 1: Layer: [OR(1, 3), AND(1, 3), AND(3, 1)]
Layer 2: Layer: [NOT(1)]

Outputs after mutation:
Sample 1: Input: [0 1 0], Output after mutation: [1]
Sample 2: Input: [1 1 1], Output after mutation: [1]
Sample 3: Input: [1 1 0], Output after mutation: [1]
Sample 4: Input: [0 0 1], Output after mutation: [1]
Sample 5: Input: [1 0 1], Output 

# Genetic Algorithm for Logic Gate Network Optimization

This section implements a Genetic Algorithm (GA) to evolve a population of logic gate networks to match a target network structure.

In [ ]:
class GeneticAlgorithm:
    def __init__(self, target_network, input_data, input_size, layer_sizes, population_size=50, mutation_rate=0.2, crossover_rate=0.5):
        self.target_network = target_network
        self.target_structure = self.net(target_network)
        self.input_data = input_data
        self.input_size = input_size
        self.layer_sizes = layer_sizes
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.population = [LogicGateNetwork(layer_sizes, input_size) for _ in range(population_size)]

    def net(self, network):
        structure = []
        for layer in network.layers:
            layer_structure = [(neuron.operator, neuron.inputs) for neuron in layer.neurons]
            structure.append(layer_structure)
        return structure

    def fitness(self, network):
        network_structure = self.net(network)
        score = 0

        for target_layer, candidate_layer in zip(self.target_structure, network_structure):
            for (target_operator, target_inputs), (candidate_operator, candidate_inputs) in zip(target_layer, candidate_layer):
                if target_operator == candidate_operator and target_inputs == candidate_inputs:
                    score += 1

        return score

    def select(self):
        fitnesses = np.array([self.fitness(network) for network in self.population])
        probabilities = np.exp(fitnesses - np.max(fitnesses))
        probabilities /= np.sum(probabilities)
        return np.random.choice(self.population, p=probabilities)

    def crossover(self, parent1, parent2):
        child = copy.deepcopy(parent1)
        for i, (layer1, layer2) in enumerate(zip(parent1.layers, parent2.layers)):
            for j, (neuron1, neuron2) in enumerate(zip(layer1.neurons, layer2.neurons)):
                if np.random.rand() < self.crossover_rate:
                    child.layers[i].neurons[j] = copy.deepcopy(neuron2)
        return child

    def mutate(self, network):
        if np.random.rand() < self.mutation_rate:
            layer_idx = np.random.randint(len(network.layers))
            neuron_idx = np.random.randint(len(network.layers[layer_idx].neurons))
            num_inputs = len(network.layers[layer_idx - 1].neurons) if layer_idx > 0 else self.input_size
            network.layers[layer_idx].neurons[neuron_idx].mutate(num_inputs=num_inputs)

    def GA(self):

        max_fitness = sum(len(layer) for layer in self.target_structure)
        generation = 0

        while True:
            new_population = []
            for _ in range(self.population_size):
                parent1 = self.select()
                parent2 = self.select()
                child = self.crossover(parent1, parent2)
                self.mutate(child)
                new_population.append(child)
            self.population = new_population

            best_network = max(self.population, key=self.fitness)
            best_fitness = self.fitness(best_network)

            if generation % 100 == 0:
                print(f"Generation {generation}, Best Fitness: {best_fitness}")

            if best_fitness == max_fitness:
                print(f"Generation: {generation}")
                return best_network

            generation += 1

## Experiment Setup and Results

- Target network initialized randomly and mutated.
- Input samples generated.
- GA runs to evolve networks that match the target.
- Prints fitness progress every 100 generations.
- Stops when an exact match is found or max fitness is reached.

In [ ]:
num_samples = 20
input_size = 4
network = LogicGateNetwork(layer_sizes=[6, 5, 3, 1], input_size=input_size)
network.mutate()

print("Target network:")
print(network)
print(" ")

inputs, target_outputs = generate_data(network, num_samples, input_size)

ga = GeneticAlgorithm(target_network=network, input_data=inputs, input_size=input_size, layer_sizes=[6, 5, 3, 1])
found_network = ga.GA()

print("\nGA network:")
print(found_network)

Target network:
Layer 0: Layer: [AND(1, 0), XOR(0, 1), AND(0, 1), XOR(0, 1), OR(1, 0), XOR(1, 0)]
Layer 1: Layer: [AND(0, 1), OR(1, 3), NOT(2), XOR(2, 1), XOR(3, 4)]
Layer 2: Layer: [XOR(3, 0), XOR(3, 4), AND(3, 4)]
Layer 3: Layer: [OR(0, 1)]
 
Generation 0, Best Fitness: 2
Generation 100, Best Fitness: 9
Generation 200, Best Fitness: 12
Generation 300, Best Fitness: 12
Generation: 393

GA network:
Layer 0: Layer: [AND(1, 0), XOR(0, 1), AND(0, 1), XOR(0, 1), OR(1, 0), XOR(1, 0)]
Layer 1: Layer: [AND(0, 1), OR(1, 3), NOT(2), XOR(2, 1), XOR(3, 4)]
Layer 2: Layer: [XOR(3, 0), XOR(3, 4), AND(3, 4)]
Layer 3: Layer: [OR(0, 1)]


### Summary

In this experiment, we aimed to reconstruct a target logic gate neural network using a genetic algorithm. The network consisted of 4 layers with various logic gates (AND, OR, XOR, NOT) as neurons.

The genetic algorithm started with a randomly initialized population of networks and evolved them over generations using selection, crossover, and mutation based on structural fitness.

- The process successfully reconstructed the **exact architecture** of the target network.
- The target structure was fully recovered by **generation 393**, with a perfect fitness score.
- This demonstrates that the genetic algorithm is effective at evolving logic-based neural architectures toward a known target structure.

The result validates the approach and shows its potential for symbolic network design using evolutionary computation.
